In [1]:
try:
    import dolfinx
except ImportError:
    !wget "https://github.com/fem-on-colab/fem-on-colab.github.io/raw/a312183/releases/fenicsx-install-real.sh" -O "/tmp/fenicsx-install.sh" && bash "/tmp/fenicsx-install.sh"
    import dolfinx

In [2]:
# try:
#     import viskex
# except ImportError:
#     !pip3 install "viskex@git+https://github.com/viskex/viskex.git@64c23fe"
#     import viskex

In [3]:
'''

'''

'\n\n'

In [4]:
try:
    import gmsh
except ImportError:
    !wget "https://github.com/fem-on-colab/fem-on-colab.github.io/raw/a312183/releases/gmsh-install.sh" -O "/tmp/gmsh-install.sh" && bash "/tmp/gmsh-install.sh"
    import gmsh

In [5]:
import basix.ufl
import dolfinx.fem
import dolfinx.fem.petsc
import dolfinx.io
import gmsh
import mpi4py.MPI
import numpy as np
import numpy.typing
import petsc4py.PETSc # PETSc (the Portable, Extensible Toolkit for Scientific Computation)
import ufl
from mpi4py import MPI

**1. Geometry & Mesh**

In [6]:
# Unit square domain [0, 1] x [0, 1] with (20,20) mesh elements
n_elements = 16
mesh = dolfinx.mesh.create_unit_square(MPI.COMM_WORLD, n_elements, n_elements)
# The resulting mesh object represents the discretized domain on which FEA will be performed
print("mesh.topology.dim = ", mesh.topology.dim)
# Create connectivities required by the rest of the code
mesh.topology.create_connectivity(mesh.topology.dim - 1, mesh.topology.dim)

mesh.topology.dim =  2


In [7]:
print("mesh.geometry.dim = ", mesh.geometry.dim)
print("mesh.basix_cell() = ", mesh.basix_cell())

# velocity space element
V_element = basix.ufl.element("Lagrange", mesh.basix_cell(), 1, shape=(mesh.geometry.dim, ))
# Lagrange elements are commonly used for continuous function spaces
# polynomial degree of the Lagrange element is 1

# tensor space element
Q_element = basix.ufl.element("Lagrange", mesh.basix_cell(), 1, shape=(mesh.geometry.dim, mesh.geometry.dim), symmetry=True)
# symmetry=True => symmetric tensor field in case of stress tensor in viscoelastic fluid models

mesh.geometry.dim =  2
mesh.basix_cell() =  CellType.triangle


**2. Function Spaces**

In [8]:
# define the function space
V = dolfinx.fem.functionspace(mesh, V_element)
Q = dolfinx.fem.functionspace(mesh, Q_element)

In [9]:
#!zip -r /content/file.zip /content/test_u.bp
#from google.colab import files
#files.download("/content/file.zip")


In [10]:
# interpolate velocity in the nodal value
def u_eval(x: np.typing.NDArray[np.float64]) -> np.typing.NDArray[  # type: ignore[no-any-unimported]
        petsc4py.PETSc.ScalarType]:
    """Return the flat velocity profile at the inlet."""
    values = np.zeros((2, x.shape[1]))
    values[0, :] = np.sin(np.pi*x[0])
    values[1, :] = -np.pi*x[1]* np.cos(np.pi*x[1])
    return values

u = dolfinx.fem.Function(V,name="velocity")
# u.interpolate(u_eval): Interpolates the u_eval function over the mesh, setting the values of u at the nodal points according to u_eval.
u.interpolate(u_eval)

with dolfinx.io.VTXWriter(mesh.comm,  f"test_u.bp", u) as vtx_u:
    vtx_u.write(0)
#https://fenicsproject.discourse.group/t/numerical-values-from-ufl-spatialcoordinate/11064/2

**3. Shear Stress Tensor**

In [11]:
def tau11_eval(x: np.typing.NDArray[np.float64]) -> np.typing.NDArray[  # type: ignore[no-any-unimported]
        petsc4py.PETSc.ScalarType]:
    """Return the flat velocity profile at the inlet."""
    values = np.zeros((1, x.shape[1]))
    values[0, :] = np.sin(np.pi*x[0])
    return values
def tau12_eval(x: np.typing.NDArray[np.float64]) -> np.typing.NDArray[  # type: ignore[no-any-unimported]
        petsc4py.PETSc.ScalarType]:
    """Return the flat velocity profile at the inlet."""
    values = np.zeros((1, x.shape[1]))
    values[0, :] = - np.pi*x[1]*np.cos(np.pi*x[0])
    return values
def tau22_eval(x: np.typing.NDArray[np.float64]) -> np.typing.NDArray[  # type: ignore[no-any-unimported]
        petsc4py.PETSc.ScalarType]:
    """Return the flat velocity profile at the inlet."""
    values = np.zeros((1, x.shape[1]))
    values[0, :] = np.sin(np.pi*x[0])*np.cos(np.pi*x[1])
    return values

def tau_eval(x: np.typing.NDArray[np.float64]) -> np.typing.NDArray[  # type: ignore[no-any-unimported]
        petsc4py.PETSc.ScalarType]:
    values = np.zeros((4, x.shape[1]))
   # using python reshape A B, B C become A B B C
    values[0, :] = tau11_eval(x)
    values[1, :] = tau12_eval(x) # tau12_eval(x)
    values[2, :] = tau12_eval(x)
    values[3, :] = tau22_eval(x)

    return values

**4. Boundary Conditions**

In [12]:
q_D = dolfinx.fem.Function(Q,name="stress")
q_D.interpolate(tau_eval)
fdim = mesh.topology.dim - 1
boundary_facets = dolfinx.mesh.locate_entities_boundary(mesh, fdim, lambda x: np.full(x.shape[1], True, dtype=bool))
print("boundary_facets = ", boundary_facets)
bc_D = dolfinx.fem.dirichletbc(q_D, dolfinx.fem.locate_dofs_topological(Q, fdim, boundary_facets))

boundary_facets =  [  0   3   4   6  12  14  23  25  37  39  54  56  74  76  97  99 123 125
 152 154 184 186 219 221 257 259 298 300 342 344 389 391 438 439 441 484
 486 526 528 565 567 601 603 634 636 664 666 691 693 715 717 736 738 754
 756 769 771 781 783 790 792 796 798 799]


**5. Trial & Test Functions**

In [13]:
psi = ufl.TestFunction(Q)
tau = ufl.TrialFunction(Q)
dx=ufl.Measure("dx", domain=mesh, metadata={"quadrature_degree": 10})
Wi = 0.9
dt = 0.0001

**6. Initial Condition**

In [14]:
# Create initial condition
q_n = dolfinx.fem.Function(Q)
q_n.name = "q_n"
dolfinx.fem.set_bc(q_n.vector, [bc_D])

with dolfinx.io.XDMFFile(mesh.comm, "initialStress.xdmf", "w") as xdmf:
    xdmf.write_mesh(mesh)
    xdmf.write_function(q_n)


In [15]:
formulation_one = False
if formulation_one:

  F = ufl.inner(tau,psi)*dx - dt*ufl.inner(ufl.dot(2*ufl.skew(ufl.grad(u)), tau),psi)*dx
  # transport part
  F+= dt*ufl.inner(ufl.dot(u,ufl.grad(tau)), psi)*dx

  # to complete here there is kappa factor..
  h = ufl.CellDiameter(mesh)
  beta = 1e-2
  stab  = beta* dt*h* ufl.inner(ufl.grad(tau),ufl.grad(psi))*dx
  F+=stab

  # see stabilization implemented PLS for the stress- need stabilization since it esplode!!
  # PhD thesis
  #https://orca.cardiff.ac.uk/id/eprint/118620/1/Theoretical%20and%20Computational%20Modelling%20of%20Compressible%20Viscoelastic%20Fluids%20-%20Alex%20Mackay%202018.pdf
  # Code
  #https://github.com/ATMackay/fenics/blob/master/lid-driven-cavity/comp_LDC.py
  L = 1/Wi *dt* (ufl.inner(ufl.Identity(mesh.geometry.dim),psi)*dx - ufl.inner(2*ufl.skew(ufl.grad(u)),psi)*dx  )
  L += 1/Wi* ufl.inner(q_n,psi)*dx

In [16]:
def ConvectStress(u, tau):
   return ufl.dot(u,ufl.grad(tau))- ufl.dot(ufl.grad(u), tau) - ufl.dot(tau, ufl.grad(u))

lhs = (Wi/dt+1)*tau + Wi* ConvectStress(u,tau)
rhs = Wi/dt* q_n + ufl.Identity(len(u))
F = ufl.inner(lhs, psi)*dx
L = ufl.inner(rhs,psi)*dx

# SUPG stabilization
h = ufl.CellDiameter(mesh)
magnitudeVel = ufl.sqrt(ufl.dot(u,u))

reg =1e-6
alpha_supg = h /(magnitudeVel+reg)
supg = ufl.inner( Wi*ufl.dot(u,ufl.grad(tau)), alpha_supg *ufl.dot(u, ufl.grad(psi)) )*dx
F += supg

In [17]:
!mkdir -p output

In [18]:
from dolfinx.fem.petsc import LinearProblem

xdmf = dolfinx.io.XDMFFile(mesh.comm, "output/stressEvolution1.xdmf", "w", encoding=dolfinx.io.XDMFFile.Encoding.ASCII)
xdmf.write_mesh(mesh)

class Velocity():

  def __init__(self, t):
    self.t = t

  def __call__(self, x):
    values = np.zeros((mesh.topology.dim , x.shape[1]), dtype=petsc4py.PETSc.ScalarType)
    values[0] = np.exp(-0.1 * self.t) * np.sin(np.pi * x[0])
    values[1] = np.pi * np.exp(-0.1 * self.t) * x[1] * np.cos(np.pi * x[0])
    return values

t = 0
velocity = Velocity(t)
u.interpolate(velocity)

xdmf.write_function(q_n,t)
num_dt = 10
problem = LinearProblem(F, L, bcs=[bc_D], petsc_options={"ksp_type": "preonly", "pc_type": "lu"})
for n in range(num_dt):
    t+=dt
    velocity.t =  t
    u.interpolate(velocity)

    qh = problem.solve()
    qh.x.scatter_forward()

    # Update solution at previous time step (u_n)
    q_n.x.array[:] = qh.x.array
    q_n.name="stress"
    xdmf.write_function(q_n,t)

xdmf.close()

In [19]:
# we need to do the static case now..

def convergence(uh,q_D):
    
  error_local = dolfinx.fem.assemble_scalar(dolfinx.fem.form((uh - q_D)**2 * ufl.dx))
  error_localH1 = dolfinx.fem.assemble_scalar(dolfinx.fem.form((uh - q_D)**2 * ufl.dx + ufl.grad(uh - q_D)**2 * ufl.dx) )

  error_L2 = numpy.sqrt(mesh.comm.allreduce(error_local, op=MPI.SUM))
  error_H1 =  numpy.sqrt(mesh.comm.allreduce(error_localH1, op=MPI.SUM))
    
  if mesh.comm.rank == 0:
      print(f"L2-error: {error_L2:.2e}")
      print(f"H1-error: {error_H1:.2e}")

  # Compute values at mesh vertices
  error_max = mesh.comm.allreduce(numpy.max(numpy.abs(uh.x.array - q_D.x.array)), op=MPI.MAX)
  if mesh.comm.rank == 0:
      print(f"Error_max: {error_max:.2e}")

convergence(qh,q_D)

L2-error: 1.98e+00
H1-error: 2.18e+01
Error_max: 2.65e+00
